In [ ]:
from slack_sdk import WebClient
SLACK_API_TOKEN = ""

client = WebClient(token=SLACK_API_TOKEN)

res = client.api_test()
if not res["ok"]:
    raise ValueError(f"Error initializing Slack API: {res['error']}")

cursor = None
result = client.conversations_history(
            channel="C04S4B91HH7",
            cursor=cursor,
        )
import pprint
pprint.pprint(result["messages"])

In [13]:
import openai
openai.api_key = 'Free the models'

# Point to leapfrogai
openai.api_base = "https://leapfrogai.leapfrogai.bigbang.dev/openai"
print(openai.Model.list())
from langchain.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(openai_api_key="foobar",
                              openai_api_base="https://leapfrogai.leapfrogai.bigbang.dev/openai",
                              model="text-embedding-ada-002")

from langchain.vectorstores import Weaviate
import weaviate
client = weaviate.Client(url="https://weaviate.leapfrogai.bigbang.dev",
                         additional_headers={
        'X-OpenAI-Api-Key': "foobar"
    })
print(client.schema.get())
print(client.get_meta())

client.schema.delete_class(class_name="Slack")
schema = {
    "classes": [
        {
            "class": "Slack",
            "description": "Things from slack",
            "vectorizer": "text2vec-transformers",
              "moduleConfig": {
                "text2vec-openai": {
                  "model": "ada",
                  "modelVersion": "002",
                  "type": "text"
                }
              },
            "properties": [
                {
                    "dataType": ["text"],
                    "description": "The content of the paragraph",
                    "moduleConfig": {
                        "text2vec-transformers": {
                          "skip": False,
                          "vectorizePropertyName": False
                        }
                      },
                    "name": "content",
                },
                {
                    "dataType": ["text"],
                    "description": "The source of the paragraph",
                    "moduleConfig": {
                        "text2vec-transformers": {
                          "skip": False,
                          "vectorizePropertyName": False
                        }
                      },
                    "name": "source",
                },
                {
                    "dataType": ["text"],
                    "description": "The slack channel",
                    "moduleConfig": {
                        "text2vec-transformers": {
                          "skip": False,
                          "vectorizePropertyName": False
                        }
                      },
                    "name": "channel",
                },
                {
                    "dataType": ["text"],
                    "description": "The Slack Channel ID",
                    "moduleConfig": {
                        "text2vec-transformers": {
                          "skip": False,
                          "vectorizePropertyName": False
                        }
                      },
                    "name": "channel_id",
                },
            ],
        },
    ]
}

client.schema.create(schema)

vectordb = Weaviate(client, "Slack", "content", embedding=embeddings)

{
  "data": [
    {
      "created": 0,
      "id": "stablelm-3b",
      "object": "",
      "owned_by": "Defense Unicorns second dynamic",
      "parent": "",
      "permission": [],
      "root": ""
    },
    {
      "created": 0,
      "id": "text-embedding-ada-002",
      "object": "",
      "owned_by": "Defense Unicorns second dynamic",
      "parent": "",
      "permission": [],
      "root": ""
    },
    {
      "created": 0,
      "id": "all-MiniLM-L6-v2",
      "object": "",
      "owned_by": "Defense Unicorns",
      "parent": "",
      "permission": [],
      "root": ""
    },
    {
      "created": 0,
      "id": "repeater",
      "object": "",
      "owned_by": "Defense Unicorns",
      "parent": "",
      "permission": [],
      "root": ""
    },
    {
      "created": 0,
      "id": "mpt-30b-chat-ggml",
      "object": "",
      "owned_by": "Defense Unicorns second dynamic",
      "parent": "",
      "permission": [],
      "root": ""
    },
    {
      "created": 0,
 

In [9]:
# download a slack channel

AI_ML_CHANNEL_ID = "C04S4B91HH7"
SLACK_API_TOKEN = ""

from llama_index import download_loader

SlackReader = download_loader("SlackReader")

loader = SlackReader(SLACK_API_TOKEN)
documents = loader.load_data(channel_ids=[AI_ML_CHANNEL_ID])

# ingest_documents()

/home/tom/Workspaces/embeddings/leapfrogai/.venv/lib/python3.10/site-packages/pkg_resources/_vendor/pyparsing.py:428: ResourceWarning: unclosed file <_io.FileIO name='/home/tom/Workspaces/embeddings/leapfrogai/.venv/lib/python3.10/site-packages/llama_index/readers/llamahub_modules/slack/requirements.txt' mode='rb' closefd=True>
  self.__tokdict[k] = self.__tokdict.get(k,list()) + [_ParseResultsWithOffset(v,0)]
Rate limit error reached, sleeping for: 10 seconds
Rate limit error reached, sleeping for: 10 seconds


In [16]:
# pull out  and clean up the links 

import re

def extract_links(string):
  """Extracts all the links from a string.

  Args:
    string: The string to extract links from.

  Returns:
    A list of links.
  """

  links = []
  pattern = r"(https?://[^|>\)\"\[\s]+)"
  matches = re.findall(pattern, string)
  for match in matches:
    links.append(match)

  return links


def clean_string(text):
    # Split the string by spaces.
    # This gives us a list where multi-spaces will be represented as ''.
    text_list = text.split(' ')

    # Rejoin with ' ' instead of ''
    cleaned_text = ''.join([' ' if x == '' else x for x in text_list ])
    # also recommended by weaviate to replace new line characters with spaces
    return cleaned_text.replace("  ", " ").replace('\n', ' ')
def percentage_of_char(input_string, char):
    count_char = input_string.count(char)
    total_chars = len(input_string)
    percentage = (count_char / total_chars) * 100
    return percentage

from llama_index import download_loader

UnstructuredURLLoader = download_loader("UnstructuredURLLoader")
links = extract_links(documents[0].text)
from langchain.text_splitter import  TokenTextSplitter
text_splitter = TokenTextSplitter(chunk_size=800, chunk_overlap=400)

# do one link at a time so we can attach the link url
for l in links:
  where_filter = {
    "path": ["source"],
    "operator": "Equal",
    "valueString": l
  }

  query_result = (
    client.query
    .get("Slack", ["content", "source"])
    .with_where(where_filter)
    .do()
  )
  if len(query_result["data"]["Get"]["Slack"]) >0:
    print(f"Found { len(query_result['data']['Get']['Slack']) } entries from URL { l } in database.  Skipping")
    continue

  print(f"Proccessing { l }:", end=" ")
  dict = {
      "channel_id": AI_ML_CHANNEL_ID,
      "channel": "#ai-ml",
      "source": l,
  }
  try:
    site = UnstructuredURLLoader([l], continue_on_failure=True, headers={"User-Agent": "leapfrogai"}).load()
  except Exception as e:
     print(f"Got an error... moving on { e }")
     continue
  print(f"Downloaded...:", end=" ")
  for t in site:
    if len(t.text) == 0:
        continue
    if percentage_of_char(t.text, ' ') > 25:
        t.text = clean_string(t.text)
    t.text = t.text.replace('\n', ' ')
    ts = text_splitter.split_text(t.text)
    print(f"{ len(ts) } parts.", end=" ")
    metadatas = [dict for t in ts ]

    ids = vectordb.add_texts(
                texts=ts,
                metadatas=metadatas,
            )
    print(f"Added to Weaivate")






/home/tom/Workspaces/embeddings/leapfrogai/.venv/lib/python3.10/site-packages/pkg_resources/_vendor/packaging/specifiers.py:635: ResourceWarning: unclosed file <_io.FileIO name='/home/tom/Workspaces/embeddings/leapfrogai/.venv/lib/python3.10/site-packages/llama_index/readers/llamahub_modules/web/unstructured_web/requirements.txt' mode='rb' closefd=True>
  return (list(itertools.chain(*left_split)), list(itertools.chain(*right_split)))


Found 10 entries from URL https://microsoft.github.io/TypeChat/blog/introducing-typechat/ in database.  Skipping
Found 10 entries from URL https://microsoft.github.io/TypeChat/blog/introducing-typechat/ in database.  Skipping
Found 2 entries from URL https://twitter.com/devgerred/status/1682106334865420289?s=46&amp;t=ejra2wzjXeM9RbsaPJEt4Q in database.  Skipping
Found 2 entries from URL https://twitter.com/devgerred/status/1682106334865420289?s=46&amp;t=ejra2wzjXeM9RbsaPJEt4Q in database.  Skipping
Found 1 entries from URL https://twitter.com/Yampeleg/status/1681409926004916246 in database.  Skipping
Found 2 entries from URL https://ai.meta.com/llama/ in database.  Skipping
Proccessing https://twitter.com/cocktailpeanut/status/1680967014888747010: Downloaded...: 1 parts. Added to Weaivate
Proccessing https://twitter.com/abacaj/status/1680979725500416001: Downloaded...: 1 parts. Added to Weaivate
Proccessing https://arxiv.org/abs/2307.07164: Downloaded...: 3 parts. Added to Weaivate
Pro

Error fetching or processing https://github.com/defenseunicorns/data-derby-weather-forecaster-serving/blob/main/pyproject.toml#L9, exeption: Expected content type text/html. Got text/plain; charset=utf-8.


Downloaded...: Proccessing https://github.com/defenseunicorns/leapfrogai/blob/main/pyproject.toml#L21: 

Error fetching or processing https://github.com/defenseunicorns/leapfrogai/blob/main/pyproject.toml#L21, exeption: Expected content type text/html. Got text/plain; charset=utf-8.


...

Downloaded...: Proccessing https://github.com/openai/tiktoken: Downloaded...: 3 parts. Added to Weaivate
Proccessing https://interconnected.org/home/2023/03/16/singularity: Downloaded...: 7 parts. Added to Weaivate
Proccessing https://hachyderm.io/@laskewitz/110033571675170703: Downloaded...: 1 parts. Added to Weaivate
Found 4 entries from URL https://scrollprize.org/ in database.  Skipping
Proccessing https://www.youtube.com/watch?v=GduCExxB0vw: Downloaded...: 1 parts. Added to Weaivate
Proccessing https://twitter.com/AlphaSignalAI/status/1635742834190958598: Downloaded...: 1 parts. Added to Weaivate
Proccessing https://cocktailpeanut.github.io/dalai/#/: Downloaded...: Proccessing https://cocktailpeanut.github.io/dalai/#/: Downloaded...: Proccessing https://stable-diffusion-art.com/how-stable-diffusion-work/: Downloaded...: 18 parts. Added to Weaivate
Proccessing https://github.com/huggingface/community-events/blob/main/keras-dreambooth-sprint/README.md: 

Error fetching or processing https://github.com/huggingface/community-events/blob/main/keras-dreambooth-sprint/README.md, exeption: Expected content type text/html. Got text/plain; charset=utf-8.


Downloaded...: Proccessing http://ann-benchmarks.com/: Downloaded...: 3 parts. Added to Weaivate
Proccessing https://huggingface.co/defenseunicorns: Downloaded...: 1 parts. Added to Weaivate
Proccessing https://huggingface.co/organizations/defenseunicorns/share/XsJgcZnSQiEddmxdzurZcnIjQrSeTIlhIW: Downloaded...: 1 parts. Added to Weaivate
Proccessing https://www.kaggle.com/: Downloaded...: Proccessing http://huggingface.co: Downloaded...: 1 parts. Added to Weaivate
Found 2 entries from URL https://huggingface.co/defenseunicorns in database.  Skipping
Found 1 entries from URL https://huggingface.co/organizations/defenseunicorns/share/XsJgcZnSQiEddmxdzurZcnIjQrSeTIlhIW in database.  Skipping
Proccessing https://www.kaggle.com/gerred: Downloaded...: Proccessing https://www.kaggle.com/nightranger77/datasets: Downloaded...: Proccessing https://www.kaggle.com/competitions/nlp-getting-started: Downloaded...: Proccessing https://www.buildt.ai/blog/vm3qozd4qfrbbyzukqhynrwm9vb9tq: Downloaded...: 

Error fetching or processing https://github.com/openai/openai-python/blob/main/chatml.md, exeption: Expected content type text/html. Got text/plain; charset=utf-8.


Added to Weaivate
Proccessing https://github.com/openai/openai-python/blob/main/chatml.md: Downloaded...: Found 17 entries from URL https://www.buildt.ai/blog/vm3qozd4qfrbbyzukqhynrwm9vb9tq in database.  Skipping


In [ ]:
bm25 = {
"query": "https://github.com/ray-project/llm-numbers",
}

# just show the elements that are from the source.  Good for seeing if the URL is already ingested
where_filter = {
  "path": ["source"],
  "operator": "Equal",
  "valueString": "https://ai.meta.com/llama/"
}

query_result = (
  client.query
  .get("Slack", ["content", "source"])
  .with_where(where_filter)
  .do()
)

import pprint
pprint.pprint(query_result)

In [17]:



# add the raw slack messages too
messages = text_splitter.split_text(documents[0].text)

for t in messages:
    t = t.replace('\n', ' ')
dict = {
    "channel_id": AI_ML_CHANNEL_ID,
    "channel": "#ai-ml",
    "source": "channel"
}


metadatas = [dict for t in messages ]
vectordb.add_texts(
            texts=messages,
            metadatas=metadatas,
        )

['6d460a3e-a13d-4289-8611-fae4b61914c4',
 '16717baf-8b29-4c0b-864e-4382ed35a2fa',
 'c80d1d33-c992-41ef-9a82-ea85f2b37895',
 '7667278d-3e0a-4150-94c0-6c046ebda77b',
 '47a7faea-3cdb-4fec-8490-761fdf8bd2c1',
 'c9d881be-025c-40d2-9fb8-e01abf0171fe',
 'e831531e-1152-4012-bf3c-3b65642c564f',
 '44b5d431-5e8d-47fd-afca-e12a6339545a',
 '8942443a-62fb-4388-a2ea-4a9b58ba0d2e',
 '656c36b2-1d09-455b-9f48-31939f0f3a29',
 'e5fea11c-561d-4d09-9519-888ccdd0fa09',
 '3d891be4-3e07-499c-9404-2c98b06541ea',
 'cbc2fcf5-cc4e-4835-8687-75cc7c186021',
 'f0062f88-b5ef-401a-b057-26d3b0d9f070',
 'ac67f532-ac7f-4d9a-a2bd-7887a8c5d95c',
 '16246b01-fe99-4eee-bbcd-ce8c6d26b4dd',
 '5b583b33-377d-4f7f-8128-daf2af372d06',
 '67e5cb07-44c9-4acb-a20c-6f7edf7cf3e6',
 'a0699907-3c5a-4d22-936b-064490d21b22',
 '2b395061-0371-464b-a18d-c9b1ed8efcfc',
 '9f142645-3869-4201-b6ad-368465ef0c00',
 '8c11ccd5-4d8e-4ce7-8cd8-dff2b86e18cf',
 '2138cbed-9cc2-432a-8e82-dee6c8a66179',
 '5942bd67-96d1-4b29-b9c4-62746d7ded73',
 '854fb61f-014d-

In [48]:
def query_weaviate(q, k=4, threshold=5):
  # query = "What LLM has the best performance for chat"


  bm25 = {
  "query": q,
  }

  result = (
    client.query
    .get("Slack", ["content","_additional {score} ", "source", "channel", "channel_id"])
    .with_bm25(**bm25)
    .with_limit(k)
    .do()
  )
  

  return result

def buildPrompt(context, query):
    prompt = f"You are a helpful AI assistant. Use the following pieces of context to answer the question at the end.\n"
    prompt += f"\n { context } "
    prompt += f"Question: { query }"
    prompt += f"Helpful answer:"
    return prompt


# prompt = f"{self.system_prompt}<|USER|>{prompt}<|ASSISTANT|>"
system_prompt = """<|im_start|>system
You are DougBot, a large language model trained by Defense Unicorns. Answer as concisely as possible.
Knowledge cutoff: 2023-07-21
Current date: 2023-07-24<|im_end|>
"""
def get_prompt(context, query) -> str:
    prompt=system_prompt
    c = f"""<|im_start|>user
Here is some data you can use to answer future questions:
{ context }<|<im_end|>
<|im_start|>assistant
Thank you.  I will use this data if it is relevent to your future questions.<|im_end|>
<|im_start|>user
{ query }<|im_end|>
<|im_start|>assistant
"""
    if context != "":
      prompt = prompt+c

    return prompt

In [70]:
query_weaviate("What is the MPT-7B-Instruct model built to do?")

{'data': {'Get': {'Slack': [{'_additional': {'score': '7.789461'},
     'channel': '#ai-ml',
     'channel_id': 'C04S4B91HH7',
     'content': "batch size of 1.  As it turns out, the full text of The Great Gatsby weighs in at just under 68k tokens. So, naturally, we had StoryWriter read The Great Gatsby and generate an epilogue. One of the epilogues we generated is in Figure 2. StoryWriter took in The Great Gatsby in about 20 seconds (about 150k words-per-minute). Due to the long sequence length, its \x9ctyping\x9d speed is slower than our other MPT-7B models, about 105 words-per-minute.Â  Even though StoryWriter was fine-tuned with a 65k context length, ALiBi makes it possible for the model to extrapolate to even longer inputs than it was trained on: 68k tokens in the case of The Great Gatsby, and up to 84k tokens in our testing.Â  MPT-7B-Instruct  LLM pretraining teaches the model to continue generating text based on the input it was provided. But in practice, we expect LLMs to treat

In [62]:
def ask(question, k=1, threshold=5, debug=False):
    related_documents = query_weaviate(question, k=k)
    
    context = ""
    for item in related_documents["data"]["Get"]["Slack"]:
        score = item["_additional"]["score"]
        message = item["content"]
        if float(score) > threshold:
            if debug:
                print(f"Using Context with score { score }:")
            context += message + "\n"
        else:
            if debug:
                print(f"Not using context with score { score }:")
        if debug:
            print(f"    { message }")
    prompt = get_prompt(context, query=question)
    if debug:
        print(prompt)
    response = openai.Completion.create(
        model="mpt-30b-chat-ggml",
        # model = "stablelm-3b",
        prompt=prompt,
        temperature=0.9,
        max_tokens=4000,
        top_p=1,
        frequency_penalty=0.0,
        presence_penalty=0.6,
        timeout=300
        )
    if debug:
        print(response)
    answer = response.choices[0].text
    # print(answer)
    # answer = answer.replace(prompt,"",1)[1:]
    # answer = answer[len(prompt):]
    # # for r in related_documents:
    # #     print(r[0].metadata)
    # print(answer)
    return answer

In [54]:
print(ask("What is the MPT-7B-Chat model built to do?",debug=True))
print(ask("What is the MPT-7B-Instruct model built to do?", debug=True))

<|im_start|>system
You are DougBot, a large language model trained by Defense Unicorns. Answer as concisely as possible.
Knowledge cutoff: 2023-07-21
Current date: 2023-07-24<|im_end|>
<|im_start|>user
Here is some data you can use to answer future questions:
7B-Chat, and MPT-7B-StoryWriter-65k+, the last of which uses a context length of 65k tokens!  Our MPT model series is:  Licensed for commercial use (unlike LLaMA).  Trained on a large amount of data (1T tokens like LLaMA vs. 300B for Pythia, 300B for OpenLLaMA, and 800B for StableLM).  Prepared to handle extremely long inputs thanks to ALiBi (we trained on up to 65k inputs and can handle up to 84k vs. 2k-4k for other open source models).  Optimized for fast training and inference (via FlashAttention and FasterTransformer)  Equipped with highly efficient open-source training code.  We rigorously evaluated MPT on a range of benchmarks, and MPT met the high quality bar set by LLaMA-7B.  Today, we are releasing the base MPT model and 

In [64]:
answer = ask("Can you compare the MPT-7B-Chat and the new llama 2 model??",debug=True, k=7, threshold=3)

Using Context with score 5.064895:
    All  Ecosystem  Integrations  Engineering  Research  Launch  Methodology  More  Link 1  Ecosystem  Integrations  Engineering  Research  Launch  Methodology  Research  by  The MosaicML NLP Team  on  May 5, 2023  Share  Introducing MPT-7B: A New Standard for Open-Source, Commercially Usable LLMs  Introducing MPT-7B, the first entry in our MosaicML Foundation Series. MPT-7B is a transformer trained from scratch on 1T tokens of text and code. It is open source, available for commercial use, and matches the quality of LLaMA-7B. MPT-7B was trained on the MosaicML platform in 9.5 days with zero human intervention at a cost of ~$200k.  Large language models (LLMs) are changing the world, but for those outside well-resourced industry labs, it can be extremely difficult to train and deploy these models. This has led to a flurry of activity centered on open-source LLMs, such as the LLaMA series from Meta, the Pythia series from EleutherAI, the StableLM serie

APIError: Invalid response object from API: '{}' (HTTP response code was 500)

In [ ]:
print(answer)

MPT-7B-Chat is a fine-tuned version of the base MPT-7B model, while Llamaconv2 is a standalone language model. The training data and use cases for these models are different, so it's hard to directly compare them. However, I can give you some general information:

MPT-7B-Chat was fine-tuned on several datasets, including CC-By-NC-SA-4.0 (non-commercial use only) datasets such as GPT-ViCUNA, HC3, Alpacalan2, Helpful and Harmless, and Evol-Instrct datasets. It has 6.7 billion parameters and was optimized for fast training and inference via FlashAttention and FastestTransformer.

Llamaconv2 is a language model that trained on 600B tokens of data, making it larger than MPT-7B-Chat. The training methodology used by Llamas' designers to produce LlaMaConv2 includes "convolutions, attention mechanisms, and multi-resolution analyses". This means that the training method may be different from MPT's training approach.


In [7]:
# from llama_index import download_loader

# UnstructuredURLLoader = download_loader("UnstructuredURLLoader")
# links_documents = UnstructuredURLLoader(, continue_on_failure=True).load()

from unstructured.partition.html import partition_html
elements = partition_html(url="https://a16z.com/2023/06/20/emerging-architectures-for-llm-applications/",headers={"User-Agent": "leapfrogai"} )
ext = "\n\n".join([str(el) for el in elements])
metadata = {"source": url}

"https://a16z.com/2023/06/20/emerging-architectures-for-llm-applications/"


NameError: name 'url' is not defined

In [ ]:

data: List[Document]
for l in links:
   try:
      new = load_file(l)
      for d in new:
         data.append(d)
   except Exception as e:
      print(f"Could not download link { l }: { e }")


In [ ]:
print(len(links_documents))

In [9]:
import openai
openai.api_key = 'Free the models'

# Point to leapfrogai
openai.api_base = "https://leapfrogai.leapfrogai.bigbang.dev/openai"
print(openai.Model.list())
from langchain.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(openai_api_key="foobar",
                              openai_api_base="https://leapfrogai.leapfrogai.bigbang.dev/openai",
                              model="all-minilm-l6-v2")


{
  "data": [
    {
      "created": 0,
      "id": "all-MiniLM-L6-v2",
      "object": "",
      "owned_by": "Defense Unicorns",
      "parent": "",
      "permission": [],
      "root": ""
    },
    {
      "created": 0,
      "id": "mpt-30b-chat-ggml",
      "object": "",
      "owned_by": "Defense Unicorns second dynamic",
      "parent": "",
      "permission": [],
      "root": ""
    },
    {
      "created": 0,
      "id": "mpt-30b-instruct-ggml",
      "object": "",
      "owned_by": "Defense Unicorns second dynamic",
      "parent": "",
      "permission": [],
      "root": ""
    },
    {
      "created": 0,
      "id": "stablelm-3b",
      "object": "",
      "owned_by": "Defense Unicorns second dynamic",
      "parent": "",
      "permission": [],
      "root": ""
    },
    {
      "created": 0,
      "id": "all-minilm-l6-v2",
      "object": "",
      "owned_by": "Defense Unicorns second dynamic",
      "parent": "",
      "permission": [],
      "root": ""
    },
    {

NameError: name 'Weaviate' is not defined

In [29]:
# client.schema.delete_class(class_name="Slack")
schema = {
    "classes": [
        {
            "class": "Slack",
            "description": "Things from slack",
            "vectorizer": "text2vec-transformers",
              "moduleConfig": {
                "text2vec-openai": {
                  "model": "ada",
                  "modelVersion": "002",
                  "type": "text"
                }
              },
            "properties": [
                {
                    "dataType": ["text"],
                    "description": "The content of the paragraph",
                    "moduleConfig": {
                        "text2vec-transformers": {
                          "skip": False,
                          "vectorizePropertyName": False
                        }
                      },
                    "name": "content",
                },
                {
                    "dataType": ["text"],
                    "description": "The source of the paragraph",
                    "moduleConfig": {
                        "text2vec-transformers": {
                          "skip": False,
                          "vectorizePropertyName": False
                        }
                      },
                    "name": "source",
                },
            ],
        },
    ]
}

client.schema.create(schema)


[Document(text='<https://twitter.com/cocktailpeanut/status/1680967014888747010>\n'
               '\n'
               '<https://twitter.com/abacaj/status/1680979725500416001>\n'
               '\n'
               '<https://arxiv.org/abs/2307.07164>\n'
               '\n'
               "maybe there's a Framer role who gets a little more weight to "
               'evidence of someone they want to frame.\n'
               '\n'
               '"blood spatters next to <@U04N76WUZ28>\'s body, and '
               '<@U0417HGT6BU>\'s wallet was discovered next to it!"\n'
               '\n'
               'yeah, basically. The AI judge would have the full context of '
               'all the prior nights, plus whatever "evidence" the other AI '
               'generated.\n'
               '\n'
               'so... mock trial\n'
               '\n'
               "So it's a new dynamic, the players need to convince and "
               'counter-convince the AI.\n'
               '\n'
       

In [10]:
          
from langchain.vectorstores import Weaviate
import weaviate
client = weaviate.Client(url="https://weaviate.leapfrogai.bigbang.dev",
                         additional_headers={
        'X-OpenAI-Api-Key': "foobar"
    })
print(client.schema.get())
print(client.get_meta())

vectordb = Weaviate(client, "Slack", "content", embedding=embeddings)



{'classes': [{'class': 'Slack', 'description': 'Information from Lack and links in slack', 'invertedIndexConfig': {'bm25': {'b': 0.75, 'k1': 1.2}, 'cleanupIntervalSeconds': 60, 'stopwords': {'additions': None, 'preset': 'en', 'removals': None}}, 'moduleConfig': {'text2vec-openai': {'model': 'ada', 'modelVersion': '002', 'type': 'text'}, 'text2vec-transformers': {'poolingStrategy': 'masked_mean', 'vectorizeClassName': True}}, 'properties': [{'dataType': ['text'], 'description': 'The content of the paragraph', 'moduleConfig': {'text2vec-transformers': {'skip': False, 'vectorizePropertyName': False}}, 'name': 'content', 'tokenization': 'word'}, {'dataType': ['text'], 'description': 'The source of the paragraph', 'moduleConfig': {'text2vec-transformers': {'skip': False, 'vectorizePropertyName': False}}, 'name': 'source', 'tokenization': 'word'}, {'dataType': ['text'], 'description': 'The channel in slack', 'moduleConfig': {'text2vec-transformers': {'skip': False, 'vectorizePropertyName': F

In [18]:
from langchain.text_splitter import TokenTextSplitter

text_splitter = TokenTextSplitter(chunk_size=800, chunk_overlap=400)

texts = []
for t in links_documents:
    print(t)
    ts = text_splitter.split_text(t.text)
    for t2 in ts:
        texts.append(t2)
print(len(texts))

for t in texts:
    t = t.replace('\n', ' ')
dict = {
    "channel_id": AI_ML_CHANNEL_ID,
    "channel": "#ai-ml",
}


metadatas = [dict for t in texts ]






Document(text='JavaScript is not available.\n\nWe’ve detected that JavaScript is disabled in this browser. Please enable JavaScript or switch to a supported browser to continue using twitter.com. You can see a list of supported browsers in our Help Center.\n\nHelp Center\n\nTerms of Service\n      Privacy Policy\n      Cookie Policy\n      Imprint\n      Ads info\n      © 2023 X Corp.\n\nSomething went wrong, but don’t fret — let’s give it another shot.\n\nTry again', doc_id='ee99e948-ee29-4636-a569-ec806296e372', embedding=None, doc_hash='2ae083ec1419e6c643369400a22351a76afb504dbbc5bad75626d9fcd9d44d38', extra_info={'source': 'https://twitter.com/cocktailpeanut/status/1680967014888747010'})
Document(text='JavaScript is not available.\n\nWe’ve detected that JavaScript is disabled in this browser. Please enable JavaScript or switch to a supported browser to continue using twitter.com. You can see a list of supported browsers in our Help Center.\n\nHelp Center\n\nTerms of Service\n      

UnexpectedStatusCodeException: Create class! Unexpected status code: 422, with response body: {'error': [{'message': 'class name "Slack" already exists'}]}.

In [31]:

client = weaviate.Client(url="https://weaviate.leapfrogai.bigbang.dev",
                         additional_headers={
        'X-OpenAI-Api-Key': "foobar"
    })
print(client.schema.get())
print(client.get_meta())
vectordb = Weaviate(client, "Slack", "content", embedding=embeddings)
vectordb.add_texts(
            texts=contents,
            metadatas=metadatas,
        )

{'classes': [{'class': 'Slack', 'description': 'Things from slack', 'invertedIndexConfig': {'bm25': {'b': 0.75, 'k1': 1.2}, 'cleanupIntervalSeconds': 60, 'stopwords': {'additions': None, 'preset': 'en', 'removals': None}}, 'moduleConfig': {'text2vec-openai': {'model': 'ada', 'modelVersion': '002', 'type': 'text'}, 'text2vec-transformers': {'poolingStrategy': 'masked_mean', 'vectorizeClassName': True}}, 'properties': [{'dataType': ['text'], 'description': 'The content of the paragraph', 'moduleConfig': {'text2vec-transformers': {'skip': False, 'vectorizePropertyName': False}}, 'name': 'content', 'tokenization': 'word'}, {'dataType': ['text'], 'description': 'The source of the paragraph', 'moduleConfig': {'text2vec-transformers': {'skip': False, 'vectorizePropertyName': False}}, 'name': 'source', 'tokenization': 'word'}], 'replicationConfig': {'factor': 1}, 'shardingConfig': {'virtualPerPhysical': 128, 'desiredCount': 1, 'actualCount': 1, 'desiredVirtualCount': 128, 'actualVirtualCount':

['2998d5de-49cc-4334-988a-dff47b4f2a1f',
 'f8fefa0c-cbaa-4733-b9fc-ef7382bc9e1d',
 '2b777d52-74c3-49f1-9d7a-67a004245124',
 '08d6f2b1-cb22-4304-b466-4177b23984cb',
 '0b75f256-67ba-4fe4-ace9-4833bc157319',
 '63aac921-6fb2-4336-90ba-1188532d19d1',
 'c3960a51-bec7-476d-be04-363de22da725',
 '4e46e895-24ce-4512-ac6b-f4f9337e8dde',
 '274b835e-a2dd-455f-8e5c-c1aa0e7b7aca',
 '4a1e42db-3613-4f8b-9fad-6f86c11b6613',
 '5b937556-761a-4771-beb6-777788b705d0',
 '1524960c-f4f7-4066-8977-646d0d46e5d2',
 'a8493f05-e1c2-4b53-b453-ef97de0df2b3',
 'e03090c7-02cb-475e-ab33-14f7bb025593',
 '1f5ad66b-a2f9-4a65-8df9-25b8ee97552f',
 '3d8cd11e-9c73-48c5-a0d8-e4c65b2ddea6',
 'a7536f8b-20db-48b7-81a0-e3840da8e692',
 '25b9061d-f2f9-40a6-9b91-19ae508a3546',
 'd63f33e2-e6ce-4f3c-b547-0ac4af8ad460',
 'b3068750-79b0-4d0e-8585-7fcebfcc8b0b',
 'db28c039-efed-4ed7-aecb-e0d78202bded',
 '5c0104d6-582f-4072-9fd2-435d71afba1a',
 'e98b6025-001f-4aec-9d08-c46fec8bbce3',
 '2e7207fe-00a4-4ba5-a014-e14e9a78ca57',
 '68b07298-7f50-

In [34]:
client.query.aggregate("Slack").with_meta_count().do()


{'data': {'Aggregate': {'Slack': [{'meta': {'count': 479}}]}}}

In [36]:
# Add the actual slack messages
messages = text_splitter.split_text(documents[0].text)

for t in messages:
    t = t.replace('\n', ' ')
dict = {
    "channel_id": AI_ML_CHANNEL_ID,
    "channel": "#ai-ml",
}


metadatas = [dict for t in messages ]
vectordb.add_texts(
            texts=messages,
            metadatas=metadatas,
        )

['4ae8bcbf-e579-4c96-8b69-a31e09b901f1',
 'd5d3e71e-40cc-448e-9575-46b98e41f7af',
 '5b86fef4-c3a4-4c70-8017-30a81b8b3690',
 'bfc37227-b24e-4b54-9d22-2fa16763dc11',
 'a1d1e309-c907-4b2a-a839-3aa3df298bea',
 '3ff02031-5b3a-4df7-9597-7e17c2fe6894',
 '0646267e-c5ca-4be6-afde-634f43170f33',
 '9df3b138-5235-4953-a340-7bca7f1c06bc',
 '3ce718e9-e671-43f1-b037-2658e8f2e704',
 '43f95483-7df3-4fe9-a9b0-2f3cd26a7adc',
 'f20f6a21-973a-4ae7-82e0-fb04ed8b93dc',
 '24c3e484-0a97-4a22-aea7-0211ee8f2812',
 '11313bdc-f082-4e48-817a-aab314fafda2',
 'bb9a8bdc-37d9-4595-8a29-e2f54fecd4e6',
 '61b74f87-d9fa-41c9-9374-a440e3116248',
 '871a159c-def0-4b62-a09e-529c1a1e309c',
 '4c507911-14d9-4144-a147-c6497e211ec2',
 '76e30501-3f72-4c4a-bf47-c28cfa583822',
 '86103d13-9d66-4d0c-88e0-3253a1a66dd5',
 'e315aaf9-7811-44f9-afa2-553c09290b8f',
 '595605a8-9235-4d28-bc71-05a68f03e7b6',
 'c90b6162-9000-4ddc-828b-5b9d9d91faea',
 '5431b7e4-6621-4244-801f-053497e446a2',
 '16728c1e-b3e4-423c-a094-443f9d0f41ba',
 '78f7f778-7781-